Cloning into 'Knee-X-ray'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 31.75 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git

fatal: destination path 'Knee-X-ray' already exists and is not an empty directory.


In [3]:
import os

root_dir = "/content/Knee-X-ray"

for root, dirs, files in os.walk(root_dir):
    level = root.replace(root_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files[:5]:
        print(f"{sub_indent}{f}")


In [ ]:
الان شما دوباره مخزن `Knee-X-ray` رو دو بار کلون کردید، ولی هنوز چیزی از ساختار پوشه‌های داخل اون نشون ندادی، پس نمی‌دونیم تصاویر دقیقاً کجا قرار دارند.

---

### ✅ لطفاً این کد را همین الان اجرا کن تا ساختار فولدرها را چاپ کنیم:

```python
import os

root_dir = "/content/Knee-X-ray"

for root, dirs, files in os.walk(root_dir):
    level = root.replace(root_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files[:5]:
        print(f"{sub_indent}{f}")
```

---

### ⏭ بعد از اجرا:

خروجی‌ای شبیه این می‌بینید:

```
Knee-X-ray/
    dataset/
        Normal/
            image001.png
        AbNormal/
            image101.png
```

وقتی این خروجی رو اینجا بفرستی، دقیق می‌گم مسیر `dataset_path` باید چی باشه و بعد بریم سراغ ساخت `Custom Dataset` + نمایش نمونه تصاویر + آموزش مدل.

بفرست تا بریم مرحله بعد.


SyntaxError: invalid character '،' (U+060C) (<ipython-input-2-0eb82e7166a9>, line 1)

In [ ]:
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git
!apt install unrar -y > /dev/null
!unrar x /content/Knee-X-ray/train.rar /content/Knee-X-ray/train > /dev/null
!unrar x /content/Knee-X-ray/val.rar /content/Knee-X-ray/val > /dev/null
!unrar x /content/Knee-X-ray/test.rar /content/Knee-X-ray/test > /dev/null


Cloning into 'Knee-X-ray'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 31.75 MiB | 11.83 MiB/s, done.
Resolving deltas: 100% (1/1), done.




In [ ]:
train_path = "/content/Knee-X-ray/train/train"
val_path = "/content/Knee-X-ray/val/val"

In [ ]:
# اگر هنوز نکردی، ابتدا گیت کلون کن
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git

# نصب unrar برای استخراج فایل‌های rar
!apt install unrar -y > /dev/null

# استخراج فایل‌های rar به فولدرهای جدا
!unrar x /content/Knee-X-ray/train.rar /content/Knee-X-ray/train > /dev/null
!unrar x /content/Knee-X-ray/val.rar /content/Knee-X-ray/val > /dev/null
!unrar x /content/Knee-X-ray/test.rar /content/Knee-X-ray/test > /dev/null


Cloning into 'Knee-X-ray'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 31.75 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (1/1), done.




In [4]:
# 📦 نصب و بارگیری اولیه
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score, f1_score
import pandas as pd
import tensorflow as tf

# 📁 بارگذاری تصاویر
def load_images(base_path):
    images, labels = [], []
    for label_name in ['Normal', 'Abnormal']:
        folder = os.path.join(base_path, label_name)
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (224, 224))
            img = cv2.equalizeHist(img)  # افزایش کنتراست
            img = img / 255.0  # نرمال‌سازی
            images.append(img.reshape(224, 224, 1))
            labels.append(0 if label_name == 'Normal' else 1)
    return np.array(images), np.array(labels)

images, labels = load_images('Knee-X-ray/dataset')

# ✅ افزایش داده‌ها
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# 🧪 تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# بخش B - مدل‌ها

# 📌 مدل‌های سنتی (SVM و Random Forest)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

svm_model = SVC()
svm_model.fit(X_train_flat, y_train)
svm_preds = svm_model.predict(X_test_flat)

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_flat, y_train)
rf_preds = rf_model.predict(X_test_flat)

# 📌 مدل CNN ساده
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
cnn_preds = (cnn_model.predict(X_test) > 0.5).astype("int32")

# 📌 یادگیری انتقالی با ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

tl_model = Model(inputs=base_model.input, outputs=output)
tl_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
tl_model.fit(X_train_rgb, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
tl_preds = (tl_model.predict(X_test_rgb) > 0.5).astype("int32")

# بخش C - ارزیابی مدل‌ها

def get_scores(y_true, y_pred):
    return {
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1-score': f1_score(y_true, y_pred)
    }

results = {
    'Model': ['SVM', 'Random Forest', 'CNN', 'ResNet50 Transfer'],
    'Precision': [],
    'Recall': [],
    'Accuracy': [],
    'F1-score': []
}

model_preds = [svm_preds, rf_preds, cnn_preds, tl_preds]
for preds in model_preds:
    scores = get_scores(y_test, preds)
    for key in scores:
        results[key].append(scores[key])

df = pd.DataFrame(results)
print("\n📊 جدول عملکرد مدل‌ها:")
print(df)

print("""
🎯 چرا F1-score؟
F1-score میانگین هماهنگ Precision و Recall است و برای داده‌های نامتوازن مناسب است.
در وظیفه پزشکی، به‌ویژه وقتی که عدم تشخیص یک بیمار (False Negative) خطرناک است،
F1-score ارزیابی دقیق‌تری از عملکرد مدل نسبت به Accuracy می‌دهد.
""")

# ✅ عدالت و اطمینان مدل:
print("""
🔒 چگونه مدل عادلانه و قابل اعتماد بسازیم؟
- استفاده از Stratified Split و Cross-Validation
- استفاده از تکنیک‌های جلوگیری از Overfitting
- تنظیم وزن کلاس‌ها یا استفاده از Oversampling
- تحلیل عملکرد مدل روی زیرگروه‌های مختلف بیماران (مثلاً سن، جنسیت)
""")



Cloning into 'Knee-X-ray'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 31.75 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (1/1), done.


FileNotFoundError: [Errno 2] No such file or directory: 'Knee-X-ray/dataset/Normal'

In [ ]:
train_path = "/content/Knee-X-ray/train/train"
val_path = "/content/Knee-X-ray/val/val"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# افزایش داده برای تصاویر آموزش (تا مدل بهتر یاد بگیره)
train_datagen = ImageDataGenerator(
    rescale=1./255,        # همه پیکسل‌ها رو بین 0 و 1 می‌کنیم
    rotation_range=10,     # چرخش تصویر تا 10 درجه
    width_shift_range=0.1, # جابجایی عرضی تا 10 درصد
    height_shift_range=0.1,# جابجایی ارتفاعی تا 10 درصد
    brightness_range=[0.8,1.2], # تغییر روشنایی
    horizontal_flip=True,  # چرخش افقی تصویر
    fill_mode='nearest'    # پر کردن جاهای خالی تصویر
)

# فقط نرمال‌سازی تصاویر اعتبارسنجی (بدون افزایش داده)
val_datagen = ImageDataGenerator(rescale=1./255)

# ساخت ژنراتور برای آموزش
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224,224), # اندازه تصاویر
    batch_size=32,         # تعداد تصویر در هر مرحله
    class_mode='binary',   # دو کلاس داریم
    color_mode='grayscale' # تصاویر سیاه و سفید هستند
)

# ساخت ژنراتور برای اعتبارسنجی
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Knee-X-ray/train/train'

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# === تحلیل مقدماتی داده‌ها ===

# مسیر تصاویر سالم و غیرسالم
normal_images = glob("Knee-X-ray/Normal/*.png")
abnormal_images = glob("Knee-X-ray/Abnormal/*.png")

print(f"تعداد تصاویر سالم: {len(normal_images)}")
print(f"تعداد تصاویر غیرسالم: {len(abnormal_images)}")

# نمایش نمونه تصویر اولیه
def show_sample(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    plt.imshow(img, cmap='gray')
    plt.title("Sample X-ray")
    plt.axis('off')
    plt.show()

show_sample(normal_images[0])


# === پیش‌پردازش: نرمال‌سازی و حذف نویز ===

def preprocess_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))

    # نرمال‌سازی: تبدیل مقادیر پیکسل به [0, 1]
    img = img / 255.0

    # فیلتر گوسی برای حذف نویز نوری
    img = cv2.GaussianBlur(img, (3, 3), 0)

    return img

# نمایش تصویر قبل و بعد پیش‌پردازش
original = cv2.imread(normal_images[0], cv2.IMREAD_GRAYSCALE)
processed = preprocess_image(normal_images[0])

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow(original, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(processed, cmap='gray')
plt.title("Processed")
plt.axis('off')
plt.show()


# === توجیه ریاضی نرمال‌سازی ===

"""
فرمول نرمال‌سازی تصویر:

    Normalized = I / 255

که در آن I مقدار پیکسل خام بین 0 تا 255 است.
این عملیات باعث پایدار شدن گرادیان‌ها در شبکه عصبی و تسریع یادگیری می‌شود.
"""


# === افزایش داده (Data Augmentation) ===

datagen = ImageDataGenerator(
    rotation_range=10,        # چرخش جزئی برای تنوع زاویه
    width_shift_range=0.05,   # جابه‌جایی عرضی
    height_shift_range=0.05,  # جابه‌جایی طولی
    zoom_range=0.1,           # بزرگ‌نمایی جزئی
    horizontal_flip=True,     # تقارن افقی
    brightness_range=[0.9, 1.1] # افزایش/کاهش روشنایی
)

# نمایش نمونه‌ای از افزایش داده
img = preprocess_image(normal_images[0])
img = np.expand_dims(img, axis=(0, -1))  # برای ورودی ImageDataGenerator

aug_iter = datagen.flow(img, batch_size=1)

plt.figure(figsize=(10, 4))
for i in range(5):
    aug_img = next(aug_iter)[0].squeeze()
    plt.subplot(1, 5, i+1)
    plt.imshow(aug_img, cmap='gray')
    plt.axis('off')
    plt.title(f"Aug {i+1}")
plt.suptitle("نمونه‌هایی از افزایش داده")
plt.show()


تعداد تصاویر سالم: 0
تعداد تصاویر غیرسالم: 0


IndexError: list index out of range

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# فرض: data_images = [N, 224, 224] و labels = [0/1]
# این داده‌ها را از مرحله A باید تولید کرده باشید.

# برای مثال ساختگی:
# data_images = np.array([...])
# labels = np.array([0, 1, 0, 1, ...])

# اضافه کردن بعد کانال برای CNN
data_images = np.expand_dims(data_images, axis=-1)

# تقسیم داده‌ها به train و test
X_train, X_test, y_train, y_test = train_test_split(data_images, labels, test_size=0.2, random_state=42, stratify=labels)


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# برای ML داده‌ها باید به صورت فلت (ویژه‌ها در یک خط) باشند
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# مقیاس‌بندی ویژگی‌ها
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

# مدل SVM
svm_model = SVC(kernel='rbf', C=1.0)
svm_model.fit(X_train_flat, y_train)
svm_preds = svm_model.predict(X_test_flat)

# مدل Random Forest
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_flat, y_train)
rf_preds = rf_model.predict(X_test_flat)

# گزارش
print("SVM Report:")
print(classification_report(y_test, svm_preds))

print("Random Forest Report:")
print(classification_report(y_test, rf_preds))



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
cnn_scores = cnn_model.evaluate(X_test, y_test)

print("CNN Accuracy:", cnn_scores[1])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'X_train' is not defined

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# تبدیل به 3 کاناله برای ورودی مدل‌های آماده
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
base_model.trainable = False  # فریز کردن لایه‌ها

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

tl_model = Model(inputs=base_model.input, outputs=output)
tl_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss='binary_crossentropy', metrics=['accuracy'])

tl_model.fit(X_train_rgb, y_train, epochs=10, batch_size=32, validation_split=0.2)
tl_scores = tl_model.evaluate(X_test_rgb, y_test)

print("Transfer Learning Accuracy:", tl_scores[1])



NameError: name 'X_train' is not defined

In [ ]:
# 🧩 مرحله 1: کلون کردن دیتاست
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git

# 🧹 مرحله 2: وارد کردن کتابخانه‌ها
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# 📂 مرحله 3: بارگذاری داده‌ها و پیش‌پردازش تصاویر
def load_images(base_path):
    images, labels = [], []
    for label_name in ['Normal', 'Abnormal']:
        folder = os.path.join(base_path, label_name)
        if not os.path.exists(folder):
            print(f"⛔ پوشه پیدا نشد: {folder}")
            continue
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (224, 224))
            img = cv2.equalizeHist(img)      # افزایش کنتراست
            img = img / 255.0                # نرمال‌سازی
            images.append(img.reshape(224, 224, 1))
            labels.append(0 if label_name == 'Normal' else 1)
    return np.array(images), np.array(labels)

images, labels = load_images('Knee-X-ray/dataset')

# 📊 مرحله 4: تقسیم داده‌ها و افزایش داده
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# 🧠 مرحله 5: مدل‌های سنتی
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

svm_model = SVC()
svm_model.fit(X_train_flat, y_train)
svm_preds = svm_model.predict(X_test_flat)

rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train_flat, y_train)
rf_preds = rf_model.predict(X_test_flat)

# 🧠 مرحله 6: شبکه عصبی کانولوشنی (CNN)
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
cnn_preds = (cnn_model.predict(X_test) > 0.5).astype("int32")

# 🧠 مرحله 7: یادگیری انتقالی با ResNet50
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

tl_model = Model(inputs=base_model.input, outputs=output)
tl_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

tl_model.fit(X_train_rgb, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
tl_preds = (tl_model.predict(X_test_rgb) > 0.5).astype("int32")

# 📈 مرحله 8: ارزیابی مدل‌ها
def get_scores(y_true, y_pred):
    return {
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1-score': f1_score(y_true, y_pred)
    }

results = {
    'Model': ['SVM', 'Random Forest', 'CNN', 'ResNet50 Transfer'],
    'Precision': [],
    'Recall': [],
    'Accuracy': [],
    'F1-score': []
}

model_preds = [svm_preds, rf_preds, cnn_preds, tl_preds]
for preds in model_preds:
    scores = get_scores(y_test, preds)
    for key in scores:
        results[key].append(scores[key])

df = pd.DataFrame(results)
print("📊 جدول عملکرد مدل‌ها:")
print(df)

# 📌 تحلیل نهایی
print("""
✅ چرا F1-score؟
F1-score برای داده‌های نامتوازن بهتر از Accuracy است، چون ترکیبی از Precision و Recall است.
در کاربرد پزشکی، عدم تشخیص بیمار (False Negative) خطرناک است، پس F1-score معیار قابل‌اتکاتری محسوب می‌شود.

✅ چطور مدل را قابل‌اعتمادتر کنیم؟
- استفاده از Cross-validation یا Stratified split
- جلوگیری از Overfitting با EarlyStopping، Dropout
- بررسی عملکرد مدل روی زیرگروه‌های مختلف جمعیت (سن، جنسیت)
- تنظیم وزن کلاس‌ها یا استفاده از SMOTE در مدل‌های سنتی
""")


fatal: destination path 'Knee-X-ray' already exists and is not an empty directory.
⛔ پوشه پیدا نشد: Knee-X-ray/dataset/Normal
⛔ پوشه پیدا نشد: Knee-X-ray/dataset/Abnormal


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# 🚀 مرحله 1: پاک‌سازی و کلون دیتاست
!rm -rf Knee-X-ray
!git clone https://github.com/F-Aghaeipoor/Knee-X-ray.git

# 📦 وارد کردن کتابخانه‌ها
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 📂 مرحله 2: یافتن مسیر پوشه‌ها
def find_dataset_path(base_dir='Knee-X-ray'):
    for root, dirs, files in os.walk(base_dir):
        if 'Normal' in dirs and 'Abnormal' in dirs:
            return root
    raise Exception("❌ پوشه‌های Normal و Abnormal پیدا نشدند.")

dataset_path = find_dataset_path()

# 🧹 مرحله 3: بارگذاری و پیش‌پردازش تصاویر
def load_images(base_path):
    images, labels = [], []
    for label_name in ['Normal', 'Abnormal']:
        folder = os.path.join(base_path, label_name)
        for img_name in os.listdir(folder):
            img_path = os.path.join(folder, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (224, 224))
            img = cv2.equalizeHist(img)
            img = img / 255.0
            images.append(img.reshape(224, 224, 1))
            labels.append(0 if label_name == 'Normal' else 1)
    return np.array(images), np.array(labels)

images, labels = load_images(dataset_path)

# 🔄 مرحله 4: تقسیم داده‌ها + افزایش داده
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# 🎯 مرحله 5: مدل‌های سنتی (SVM + RandomForest)
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

svm = SVC()
svm.fit(X_train_flat, y_train)
svm_preds = svm.predict(X_test_flat)

rf = RandomForestClassifier()
rf.fit(X_train_flat, y_train)
rf_preds = rf.predict(X_test_flat)

# 🧠 مرحله 6: شبکه CNN ساده
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
cnn_preds = (cnn.predict(X_test) > 0.5).astype("int32")

# 🧠 مرحله 7: یادگیری انتقالی با ResNet50
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)

base = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
base.trainable = False
x = GlobalAveragePooling2D()(base.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

resnet_model = Model(inputs=base.input, outputs=output)
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
resnet_model.fit(X_train_rgb, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stop])
resnet_preds = (resnet_model.predict(X_test_rgb) > 0.5).astype("int32")

# 📈 مرحله 8: گزارش ارزیابی
def get_scores(y_true, y_pred):
    return {
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1-score': f1_score(y_true, y_pred)
    }

results = {
    'Model': ['SVM', 'Random Forest', 'CNN', 'ResNet50 Transfer'],
    'Precision': [],
    'Recall': [],
    'Accuracy': [],
    'F1-score': []
}
for preds in [svm_preds, rf_preds, cnn_preds, resnet_preds]:
    scores = get_scores(y_test, preds)
    for key in scores:
        results[key].append(scores[key])

df = pd.DataFrame(results)
print("📊 ارزیابی نهایی مدل‌ها:\n")
print(df)

# 📝 توضیح نهایی
print("""
✅ چرا F1-score مهم است؟
F1-score هم تعادل Precision و هم Recall را درنظر می‌گیرد. در مسائل پزشکی مانند تشخیص بیماری، اگر فقط دقت یا فقط Recall مهم باشد، ممکن است بیمار واقعی تشخیص داده نشود یا فرد سالم به اشتباه بیمار اعلام شود.

✅ چطور مدل را قابل اعتمادتر کنیم؟
- استفاده از Stratified Split
- EarlyStopping و Dropout برای جلوگیری از Overfitting
- بررسی عملکرد روی زیرگروه‌ها (مانند جنسیت یا سن)
- افزایش داده‌ها با حفظ ویژگی‌های واقعی تصاویر پزشکی
""")


Cloning into 'Knee-X-ray'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 31.75 MiB | 9.83 MiB/s, done.
Resolving deltas: 100% (1/1), done.


Exception: ❌ پوشه‌های Normal و Abnormal پیدا نشدند.

In [ ]:
!ls -R Knee-Xray


ls: cannot access 'Knee-Xray': No such file or directory
